In [2]:
## Imports

# utility modules
import glob
import os
import sys
import re
import rasterio as rio


# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs

# specialty modules
import h5py
import pyproj

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
cd

/home/jovyan


In [4]:
cd Land_Ice_Applications/readers

/home/jovyan/Land_Ice_Applications/readers


In [5]:
from read_HDF5_ATL03 import read_HDF5_ATL03
from get_ATL03_x_atc import get_ATL03_x_atc

### 1.2.2 Data used
Data for this tutorial are stored in a shared drive, accessible to all tutorial participants.  If you're getting data for yourself, you'll need to put it in a consistent place, and change this cell to match your directory.

In [5]:
data_root='shared/FirnAndMelt/ATL03/Antarctica/'

In [6]:
import cartopy.crs as ccrs
from rasterio import plot

In [7]:
cd

/home/jovyan


In [8]:
cd FirnAndMelt/notebooks/

/home/jovyan/FirnAndMelt/notebooks


In [9]:
landsat8 = ('LC08_L1GT_127111_20200120_20200128_01_T2_All_Masks.tif')


In [10]:
#This is rasterio Dataset object
l8_src = rio.open(landsat8)

In [11]:
l8_src.profile

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 7291, 'height': 7381, 'count': 1, 'crs': CRS.from_epsg(3031), 'transform': Affine(30.0, 0.0, 1711785.0,
       0.0, -30.0, 802515.0), 'blockxsize': 736, 'blockysize': 752, 'tiled': True, 'compress': 'packbits', 'interleave': 'band'}

In [12]:
l8_extent = rio.plot.plotting_extent(l8_src)

In [13]:
l8_m = l8_src.read(1, masked=True)

In [14]:
f, ax = plt.subplots()
ax.imshow(l8_m, extent=l8_extent, cmap='coolwarm');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

I've  also added a Landsat 8 image from 2020-01-20 that overlaps the area of interest on the Amery Ice Shelf. I've used Moussavi et al (2020)'s automated lake detection algorithm to identify lake pixels. 

In [15]:
%matplotlib widget
L8lakes = l8_m==1


# 2. Data in along-track format

# 2.1 ATL03 elevations

Before we start looking at the ATL06 data we've downloaded, let's have a look at some of the ATL03 data that were used to make them.  One of the source ATL03 files is in the shared folder, and we'll read it with Tyler Sutterley's excellent "read_HDF5_ATL03" function.

In [6]:
cd 

/home/jovyan


In [7]:
cd 'shared/FirnAndMelt/ATL03/Antarctica/'

/srv/shared/FirnAndMelt/ATL03/Antarctica


In [8]:
# read the data:
#rgt="0027"
#cycle="06"
# read the IS2 data with Tyler's ATL03 reader:
ATL03_file=  'processed_ATL03_20200203123745_05920612_003_01.h5'
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03(ATL03_file)

The data are returned in a set of dictionaries that mimic the structure of an ATL03 file.  To help visualize the data, we're going to calculate an along-track coordinate for every photon in the cloud (x_atc).  This is a slightly complex job, and there's a helper function in the readers directory that you can look at if you want the details.

In [9]:
# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)

In [10]:
IS2_atl03_mds['gt2r']

{'heights': {'delta_time': array([65968768.88493214, 65968768.88493214, 65968768.88493214, ...,
         65968780.56383242, 65968780.56383242, 65968780.56383242]),
  'dist_ph_across': array([-52.62856 , -52.638065, -52.626698, ..., -59.818478, -59.819874,
         -59.857765], dtype=float32),
  'dist_ph_along': array([ 0.48488307,  0.4367691 ,  0.49435243, ..., 19.510641  ,
         19.503958  , 19.32308   ], dtype=float32),
  'h_ph': array([197.02295, 179.9768 , 200.36516, ..., 104.63813, 102.27638,
          38.30188], dtype=float32),
  'lat_ph': array([-72.49992335, -72.49992377, -72.49992327, ..., -71.76736058,
         -71.76736064, -71.7673622 ]),
  'lon_ph': array([68.32057214, 68.32057261, 68.32057205, ..., 67.99997189,
         67.99997195, 67.99997371]),
  'pce_mframe_cnt': array([72830108, 72830108, 72830108, ..., 72830692, 72830692, 72830692],
        dtype=uint32),
  'ph_id_channel': array([90, 87, 26, ..., 24, 35, 28], dtype=uint8),
  'ph_id_count': array([1, 1, 1, ..., 2

In [11]:
df = pd.DataFrame(data={'height':IS2_atl03_mds['gt2r']['heights']['h_ph'],
                        'lat':  IS2_atl03_mds['gt2r']['heights']['lat_ph'],
                        'lon':  IS2_atl03_mds['gt2r']['heights']['lon_ph'],
                        'distance_along': IS2_atl03_mds['gt2r']['heights']['dist_ph_along'],
                        'signal_conf': IS2_atl03_mds['gt2r']['heights']['signal_conf_ph'][:,3] >= 2
                       },
                  index=IS2_atl03_mds['gt2r']['heights']['delta_time'])
df

,height,lat,lon,distance_along,signal_conf
6.596877e+07,197.022949,-72.499923,68.320572,0.484883,False
6.596877e+07,179.976807,-72.499924,68.320573,0.436769,False
6.596877e+07,200.365158,-72.499923,68.320572,0.494352,False
6.596877e+07,172.272217,-72.499924,68.320573,0.414983,False
6.596877e+07,167.983429,-72.499924,68.320573,0.402877,False
...,...,...,...,...,...
6.596878e+07,104.653831,-71.767361,67.999972,19.510685,True
6.596878e+07,102.633972,-71.767361,67.999972,19.504976,True
6.596878e+07,104.638130,-71.767361,67.999972,19.510641,True
6.596878e+07,102.276382,-71.767361,67.999972,19.503958,True


### 2.1.1 Plotting ATL03 photons
Now let's plot the ATL03 photons.  We'll plot all the photon heights as small black dots, then plot the photons that the ATL03 land-ice signal finder designates as surface (with low, medium, or high confidence) in green.  

In [22]:
#-- select the 2l beam from ATL03
D3 = IS2_atl03_mds['gt2r']

#-- create scatter plot of photon data (e.g., photon elevation vs x_atc)
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
#ax.plot(D3['heights']['lat_ph'], D3['heights']['h_ph'],'k.',markersize=0.25, label='all photons')
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
ax.plot(D3['heights']['lat_ph'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
h_leg=ax.legend()

ax.set_xlabel('lat (deg)')
ax.set_ylabel('h, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
dfg  = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']), crs='EPSG:4326')

In [24]:
dfg_PS = dfg.to_crs(l8_src.crs)

In [25]:
atl03_coord = [(pt.x, pt.y) for pt in dfg_PS.geometry]


In [26]:
atl03_m_sample = l8_src.sample(atl03_coord)


In [27]:
ph_elev = np.fromiter(atl03_m_sample, dtype=l8_m.dtype)

In [28]:
l8_elev_ma = np.ma.masked_equal(ph_elev, l8_src.nodata)


In [29]:
#add classification to photon data
dfg_PS['l8_classification'] = l8_elev_ma.astype(float).filled(np.nan)

In [30]:
dfg_PS['lake'] = dfg_PS['l8_classification']==1

In [13]:
dfg_sel = dfg[dfg['signal_conf']==True]

In [50]:
import xarray as xr

In [51]:
%matplotlib widget
ax = plt.axes(projection=ccrs.SouthPolarStereo())
Landsat8imag = xr.open_rasterio('FirnAndMelt/notebooks/lores_20200120_127111_tif.tif')
Landsat8imag.plot(ax=ax, cmap='Greys_r', transform=ccrs.SouthPolarStereo())
plt.scatter(x=dfg_sel['lon'],y=dfg_sel['lat'],c=dfg_sel['lake'], marker='.', transform=ccrs.PlateCarree());
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RasterioIOError: FirnAndMelt/notebooks/lores_20200120_127111_tif.tif: No such file or directory

In [14]:
#bimodal_coeff =  (dfg_sel['height'].rolling(100).skew()**2+1)/dfg_sel['height'].rolling(100).kurt()
max_outlier = (dfg_sel['height'].rolling(100).max()-dfg_sel['height'].rolling(100).mean())/dfg_sel['height'].rolling(100).std()
min_outlier = (dfg_sel['height'].rolling(100).mean()-dfg_sel['height'].rolling(100).min())/dfg_sel['height'].rolling(100).std()

variance =  dfg_sel['height'].rolling(100).var()

fig, ax  = plt.subplots(nrows=3, figsize=(10,10))
ax[0].plot(dfg_sel['lat'].rolling(100).mean(),dfg_sel['height'].rolling(100).var())
ax[0].set_title('Variance')
ax[1].plot(dfg_sel['lat'].rolling(100).mean(),max_outlier)
ax[1].set_title('Grubb`s Test for Outliers')
ax[2].plot(dfg_sel['lat'],dfg_sel['height']);
ax[2].set_title('Photon Height (m)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Photon Height (m)')

In [57]:
fig, ax = plt.subplots()

ax.scatter(min_outlier,max_outlier,s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
from sklearn.cluster import KMeans
import sklearn.metrics 
from sklearn.metrics import pairwise_distances


In [16]:
def get_silhouette_score(X):

    kmeans = KMeans(n_clusters=2).fit(Xdist)
    labels = kmeans.labels_
    return sklearn.metrics.silhouette_samples(X,labels, metric='euclidean')
     
cluster_score = lambda x: get_silhouette_score(x)



In [17]:
height_array = dfg_sel['height'].values
dist_array = dfg_sel['distance_along'].values


In [ ]:
X = np.array((height_array, dist_array)).T
Xdist  = sklearn.metrics.pairwise_distances(X, metric='manhattan')

In [82]:
silhouette = [0]
win_size = 100
for i in np.arange(int(winsize/2),int(len(height_array)-win_size/2)):
    window = height_array[i-50:i+50]
    score  = get_silhouette_score(window)
    silhouette  = np.append(silhouette, score)

KeyboardInterrupt: 

### 2.1.2 Cloudy tracks and singnal-finding blunders

In [14]:

TEP=(D3['heights']['signal_conf_ph'][:,3] <-1)   
BG=(D3['heights']['signal_conf_ph'][:,3] ==0)   |  (D3['heights']['signal_conf_ph'][:,3] ==1)
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2


In [15]:
%matplotlib widget
plt.plot(D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …